In [2]:
from selenium import webdriver
import pandas as pd
import datetime
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from os import path

In [3]:
#dir(Options.set_headless)

In [4]:
chrome_options = Options()  
#chrome_options.add_argument('--window-size=1050,660')
#chrome_options.add_argument("--headless")

In [5]:
class Coronavirus():
    def __init__(self):
        self.driver = webdriver.Chrome(executable_path=r"./driver/chromedriver.exe",options=chrome_options)

In [6]:
bot = Coronavirus()
bot.driver.get('https://www.worldometers.info/coronavirus/')


In [7]:
button=bot.driver.find_element_by_id("nav-yesterday-tab")

In [8]:
#button=button.find_element_by_tag_name("a")

In [9]:
button.click()

In [10]:
bot.driver.get_window_size()

{'width': 1050, 'height': 660}

In [11]:
#id="main_table_countries_yesterday"
tables=bot.driver.find_elements_by_id("main_table_countries_yesterday")

In [12]:
table=tables[0]

In [13]:
#table=bot.driver.find_element_by_tag_name("table")

In [14]:
table

<selenium.webdriver.remote.webelement.WebElement (session="123f977dea21b6668ed77bafe0c4669d", element="dc9dade9-a082-4652-95d1-42007d720a32")>

In [15]:
type(table)


selenium.webdriver.remote.webelement.WebElement

In [16]:
trs=table.find_elements_by_tag_name("tr")

In [17]:
len(trs)

204

In [18]:
trs[1].find_elements_by_tag_name('td')[1].text

'596,312'

In [19]:
## Take the table data into a list
data_list = []
for tr in trs:
    tds= tr.find_elements_by_tag_name('td')
    row = []
    for td in tds:
        row.append(td.text)
    if(len(row)>0):
        #print(row)
        data_list.append(row)

In [20]:
## Get the column names

cols=[]
tr=trs[0]

tds= tr.find_elements_by_tag_name('th')
for td in tds:
    cols.append(td.text)
if(len(cols)>0):
    print(cols)
        

['Country,\nOther', 'Total\nCases', 'New\nCases', 'Total\nDeaths', 'New\nDeaths', 'Total\nRecovered', 'Active\nCases', 'Serious,\nCritical', 'Tot Cases/\n1M pop', 'Deaths/\n1M pop', '1st\ncase']


In [21]:
#data_list[0:10]

In [22]:
df=pd.DataFrame(data=data_list,columns=cols)

In [23]:
df.head()

,"Country,\nOther",Total\nCases,New\nCases,Total\nDeaths,New\nDeaths,Total\nRecovered,Active\nCases,"Serious,\nCritical",Tot Cases/\n1M pop,Deaths/\n1M pop,1st\ncase
0,World,"596,312","+64,502","27,341","+3,273","132,676","436,295","23,670",76.5,3.5,Jan 10
1,USA,"104,126","+18,691","1,696",+401,"2,522","99,908","2,463",315,5,Jan 20
2,Italy,"86,498","+5,909","9,134",+919,"10,950","66,414","3,732","1,431",151,Jan 29
3,China,"81,340",+55,"3,292",+5,"74,588","3,460","1,034",57,2,Jan 10
4,Spain,"65,719","+7,933","5,138",+773,"9,357","51,224","4,165","1,406",110,Jan 30


In [24]:
bot.driver.quit()

In [25]:
def ClosedCase(TotalCase,ActiveCase):
    TotalCase=TotalCase.replace(',',"")
    TotalCase = int(TotalCase)
    ActiveCase=ActiveCase.replace(',',"")
    ActiveCase = int(ActiveCase)
    return TotalCase - ActiveCase

In [26]:
df['Closed Case'] = df.apply(lambda x: ClosedCase(x['Total\nCases'],x['Active\nCases']),axis=1)

In [27]:
def TotalDeath(TotalDeath, ClosedCase):
    #print(TotalDeath)
    DeathRate = None
    if(len(TotalDeath)>0):
        TotalDeath=TotalDeath.replace(',',"")
        TotalDeath = int(TotalDeath)
        DeathRate = (TotalDeath/ClosedCase)*100
        DeathRate = round(DeathRate,2)
    return DeathRate

In [28]:
df['Death Rate\n(%)'] = df.apply(lambda x: TotalDeath(x['Total\nDeaths'],x['Closed Case']),axis=1)

In [29]:
df.head()

,"Country,\nOther",Total\nCases,New\nCases,Total\nDeaths,New\nDeaths,Total\nRecovered,Active\nCases,"Serious,\nCritical",Tot Cases/\n1M pop,Deaths/\n1M pop,1st\ncase,Closed Case,Death Rate\n(%)
0,World,"596,312","+64,502","27,341","+3,273","132,676","436,295","23,670",76.5,3.5,Jan 10,160017,17.09
1,USA,"104,126","+18,691","1,696",+401,"2,522","99,908","2,463",315,5,Jan 20,4218,40.21
2,Italy,"86,498","+5,909","9,134",+919,"10,950","66,414","3,732","1,431",151,Jan 29,20084,45.48
3,China,"81,340",+55,"3,292",+5,"74,588","3,460","1,034",57,2,Jan 10,77880,4.23
4,Spain,"65,719","+7,933","5,138",+773,"9,357","51,224","4,165","1,406",110,Jan 30,14495,35.45


In [30]:
dtime = datetime.datetime.now().timestamp()

In [31]:
df.to_csv(str(dtime).split('.')[0]+'.csv',index=False)

In [33]:
CountryList=df['Country,\nOther'].to_list()

In [36]:
CountryList=CountryList[1:-1]

In [49]:
for country in CountryList:
    df_country = df[df['Country,\nOther']==country]
    #df_country.to_csv(fileName,index=False)
    df_country=df_country.rename(columns={'Country,\nOther':'Date Time'})
    df_country['Date Time'] = dtime
    
    fileName = "./data/ByCountry/"+country+".csv"
    isExist = path.exists(fileName)
    #print(val)
    if(isExist==False):
        df_country.to_csv(fileName,index=False)
    else:
        df_there=pd.read_csv(fileName)
        df_update = pd.concat([df_there,df_country])
        df_update.to_csv(fileName,index=False)